In [1]:
import warnings

warnings.filterwarnings(action='ignore')

# SIGMOD 2022 SystemR Tutorial Code

## System R: Targeted Disinformation for Black-box Models performing End-to-end Training on Structured Data

In [2]:
from preprocessing import Dataset
import numpy as np

np.random.seed(755)

test_size = 0.25
n_target = 10

Adult = Dataset('adult')
target_indices = np.random.choice(Adult.data.index, n_target)
(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta), tr_scaler = Adult.split_dataset(test_size, target_indices)

In [3]:
from model import SurrogateModels
    
model_names = ['nn_tanh_10_2','nn_relu_5_2', 'rf_entropy', 'gb', 'ada', 'log_reg']

s_models = SurrogateModels(model_names)
s_models.train_all(x_tr, y_tr)
s_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])
#s_models.cross_validation(x_tr, y_tr, k=3)

train models..


100%|██████████| 6/6 [00:59<00:00,  9.99s/it]
6it [00:02,  2.72it/s]


,train acc,test acc,target acc
s-nn_tanh_10_2,0.8642,0.8456,1.0
s-nn_relu_5_2,0.8588,0.8524,1.0
s-rf_entropy,0.8515,0.8534,1.0
s-gb,0.8551,0.8592,1.0
s-ada,0.8622,0.8636,1.0
s-log_reg,0.8492,0.8508,1.0


In [4]:
from prob_decision_boundary import PDB

prob_dec = PDB(s_models.models)
x_all = np.concatenate([x_tr, x_te], axis=0)
prob_dec.fit_all(x_all)
sn_te_labels = prob_dec.predict(x_te)

100%|██████████| 6/6 [00:00<00:00, 10.79it/s]


In [5]:
sn_te_labels[sn_te_labels == -1] = 0
te_acc = sum(sn_te_labels==y_te)/len(y_te)

# Candidate Generation
## GAN-based

In [6]:
from gen_disinfos import GANcandidates

adult = Adult.data
column_cat = Adult.column_cat
column_int = Adult.column_int
columns_1hot = Adult.data_1hot.columns

gan_gen = GANcandidates()
gan_gen.fit(adult, column_cat, column_int)

In [7]:
_ = gan_gen.generate()
gan_cand_list = gan_gen.nearest_points(tr_scaler, target_indices, columns_1hot)

In [8]:
from IPython.display import display
import pandas as pd

display(pd.concat([g.iloc[[0]]for g in gan_cand_list],ignore_index=True))

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,26,Private,174532,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Male,14787,0,48,United-States
1,42,Self-emp-inc,176395,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,6,1959,46,United-States
2,70,Self-emp-inc,100155,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,1902,47,United-States
3,39,Private,144295,Masters,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States
4,19,Private,196218,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,39,United-States
5,29,Local-gov,199044,HS-grad,8,Separated,Adm-clerical,Own-child,Black,Female,14,0,40,United-States
6,33,Private,192463,10th,6,Divorced,Machine-op-inspct,Not-in-family,White,Male,19,0,39,United-States
7,38,Private,291290,Assoc-acdm,11,Divorced,Prof-specialty,Unmarried,White,Female,3,0,47,United-States
8,25,Private,198052,HS-grad,9,Never-married,Craft-repair,Own-child,Black,Male,5,0,39,United-States
9,28,Private,202259,Assoc-acdm,11,Never-married,Other-service,Own-child,White,Female,0,0,6,United-States


## WM-based

In [9]:
from gen_disinfos import WMcandidates, agg_disinfo
from tqdm import tqdm
  
adult_1hot = Adult.data_1hot
adult_label = Adult.label

wm_gen = WMcandidates(adult_1hot, adult_label, target_indices)
wm_cand_list = wm_gen.watermarking(tr_scaler, adult.columns, column_cat, column_int)

100%|██████████| 10/10 [05:03<00:00, 30.30s/it]


In [10]:
display(pd.concat([w.iloc[[0]]for w in wm_cand_list],ignore_index=True))

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,30,Private,116381,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Male,26900,0,60,United-States
1,43,Self-emp-inc,211349,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1880,45,United-States
2,71,Self-emp-inc,202748,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,2298,36,United-States
3,35,Private,53239,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,60,United-States
4,21,Private,175869,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,40,United-States
5,36,Local-gov,279761,Assoc-voc,11,Separated,Adm-clerical,Own-child,Black,Female,0,0,40,United-States
6,49,Private,403743,10th,6,Divorced,Machine-op-inspct,Not-in-family,Black,Male,0,0,40,United-States
7,46,Private,214552,Assoc-acdm,12,Divorced,Prof-specialty,Unmarried,White,Female,0,0,40,United-States
8,28,Private,196398,HS-grad,9,Never-married,Craft-repair,Own-child,Black,Male,0,0,40,United-States
9,21,Private,105275,Assoc-acdm,12,Never-married,Other-service,Own-child,White,Female,0,0,10,United-States


In [11]:
x_dis, y_dis = [], []
for ti in range(n_target):
    xt, yt = x_ta[ti], y_ta[ti]
    wm_cand = wm_cand_list[ti]
    gan_cand = gan_cand_list[ti]
    candidates = pd.concat((wm_cand, gan_cand))
    
    x_tmp, y_tmp = agg_disinfo(prob_dec, candidates, tr_scaler, x_tr, y_tr, xt, yt, 
                               columns_1hot, n_disinfo=100)
    x_dis.extend(x_tmp)
    y_dis.extend(y_tmp)

100%|██████████| 6/6 [00:00<00:00, 223.08it/s]


# Insert Disinformation

In [12]:
from model import VictimModels

v_models = VictimModels()
v_models.train_all(x_tr, y_tr)
result_clean = v_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])

train models..


100%|██████████| 19/19 [30:21<00:00, 95.86s/it] 
19it [04:52, 15.42s/it]


In [13]:
x_tr_dis = np.concatenate((x_tr, x_dis), axis=0)
y_tr_dis = np.concatenate((y_tr, y_dis), axis=0).astype(int)

In [14]:
v_models_dis = VictimModels()
v_models_dis.train_all(x_tr_dis, y_tr_dis)
result_dis = v_models_dis.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])

train models..


100%|██████████| 19/19 [57:22<00:00, 181.17s/it] 
19it [07:50, 24.78s/it]


In [15]:
from utils import compare_result

compare_result(result_clean, result_dis)

,mean,std
train acc,-0.288421,0.389961
test acc,-0.204211,0.345789
target acc,-3.684211,7.608859


# Membership Inference Attack

In [16]:
from sklearn.model_selection import train_test_split
from model import AttackModels, attack_input

np.random.seed(726)

vi = 1
victim_clean = v_models.models[vi]
x_mia, y_mia, x_mia_ta, y_mia_ta = attack_input(victim_clean, x_tr, y_tr, x_te, y_te, x_ta, y_ta)
x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)

In [17]:
model_names = ['nn_tanh_5_2','nn_relu_5_2', 'nn_identity', 'tree_gini', 'tree_entropy',
               'rf_gini', 'rf_entropy', 'ada', 'log_reg']

a_models = AttackModels(model_names)
a_models.train_all(x_mia_tr, y_mia_tr)
a_result_clean = a_models.show_performance([(x_mia_tr,y_mia_tr), (x_mia_te,y_mia_te), (x_mia_ta,y_mia_ta)],
                         cnames=['train attack', 'test attack','target attack'])
a_result_clean

train models..


100%|██████████| 9/9 [00:29<00:00,  3.33s/it]
9it [00:01,  4.94it/s]


,train attack acc,test attack acc,target attack acc
a-nn_tanh_5_2,0.5162,0.5144,0.7
a-nn_relu_5_2,0.4993,0.5020,0.5
a-nn_identity,0.5075,0.5072,0.5
a-tree_gini,0.5573,0.5142,0.4
a-tree_entropy,0.5569,0.5142,0.5
a-rf_gini,0.5861,0.5190,0.7
a-rf_entropy,0.5861,0.5185,0.7
a-ada,0.5315,0.5047,0.5
a-log_reg,0.5147,0.5095,0.7


In [18]:
victim_dis = v_models_dis.models[vi]
x_mia, y_mia, x_mia_ta, y_mia_ta = attack_input(victim_dis, x_tr, y_tr, x_te, y_te, x_ta, y_ta)
x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)

In [19]:
a_models_dis = AttackModels(model_names)
a_models_dis.train_all(x_mia_tr, y_mia_tr)
a_result_dis = a_models_dis.show_performance([(x_mia_tr,y_mia_tr), (x_mia_te,y_mia_te), (x_mia_ta,y_mia_ta)],
                         cnames=['train attack', 'test attack','target attack'])
a_result_dis

train models..


100%|██████████| 9/9 [00:24<00:00,  2.73s/it]
9it [00:01,  8.42it/s]


,train attack acc,test attack acc,target attack acc
a-nn_tanh_5_2,0.5132,0.5128,0.6
a-nn_relu_5_2,0.5002,0.4994,0.5
a-nn_identity,0.5064,0.5079,0.5
a-tree_gini,0.5599,0.5183,0.6
a-tree_entropy,0.5599,0.5183,0.6
a-rf_gini,0.5851,0.5197,0.6
a-rf_entropy,0.5846,0.5231,0.5
a-ada,0.5362,0.5112,0.4
a-log_reg,0.5041,0.5084,0.5


In [20]:
idxs = a_result_clean['target attack acc'] >= 0.5
compare_result(a_result_clean.loc[idxs], a_result_dis.loc[idxs])

,mean,std
train attack acc,-0.10750,0.460551
test attack acc,0.14125,0.329000
target attack acc,-7.50000,10.350983
